# Global registration with RANSAC
We are going to use open3d (http://www.open3d.org/) to handle  pointclouds and generation of pointclouds


In [1]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = True):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    
    o3d.visualization.draw_geometries([source_temp, target_temp],
#                                       zoom=0.3412,
#                                       front=[0.4257, -0.2125, -0.8795],
#                                       lookat=[2.6172, 2.0475, 1.532],
#                                       up=[-0.0694, -0.9768, 0.2024]
                                     )

We need to read in our pointclouds.

### Finding features in pointclouds
When working on point clouds it can be benefitial work on a downsampled version of the point cloud.
you can use [```pointcloudname.voxel_down_sample()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html) where pointcloud is the name of your point cloud object.

Voxel downsampling uses a regular voxel grid to create a uniformly downsampled point cloud from an input point cloud. It is often used as a pre-processing step for many point cloud processing tasks. The algorithm operates in two steps:

    1. Points are bucketed into voxels.
    2. Each occupied voxel generates exactly one point by averaging all points inside.




We also need to estimate the normals of the pointcloud points using [```pointcloudname.estimate_normals()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html)

    estimate_normals: computes the normal for every point. The function finds adjacent points and calculates the principal axis of the adjacent points using covariance analysis.
    
    The function takes an instance of KDTreeSearchParamHybrid class as an argument. The two key arguments radius = 0.1 and max_nn = 30 specifies search radius and maximum nearest neighbor. It has 10cm of search radius, and only considers up to 30 neighbors to save computation time.

And finally find fpfh features or correspondance of the downsampled point clouds.
[```o3d.pipelines.registration.compute_fpfh_feature()```](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.compute_fpfh_feature.html)


In [2]:
####
# Downsample and find features here
####
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
    source = o3d.io.read_point_cloud("ICP/r1.pcd")
    target = o3d.io.read_point_cloud("ICP/r3.pcd")
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registrations(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    draw_registrations(source_down, target_down, np.identity(4))
    
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [3]:
voxel_size = 0.05 # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [5]:
# Estimated normal vectors can be retrieved from the normals variable of downpcd.
print("Print a normal vector of the 0th point")
print(source_down.normals[0])
# Normal vectors can be transformed as a numpy array using np.asarray.
print("Print the normal vectors of the first 10 points")
print(np.asarray(source_down.normals)[:10, :])
# To check out other variables, please use help(downpcd)
# help(source_down)

Print a normal vector of the 0th point
[-0.00179454 -0.98362967 -0.1801928 ]
Print the normal vectors of the first 10 points
[[-1.79453577e-03 -9.83629674e-01 -1.80192798e-01]
 [-9.87445606e-01  1.15696826e-01 -1.07542643e-01]
 [-5.74133348e-02 -9.55844085e-01 -2.88211371e-01]
 [ 3.11833525e-02 -9.98178981e-01 -5.16364159e-02]
 [ 4.13321781e-01 -3.78813730e-01 -8.28049071e-01]
 [ 3.71122831e-02 -9.68741831e-01 -2.45279316e-01]
 [-6.51199096e-02 -4.58519659e-01  8.86295165e-01]
 [-3.82736451e-03 -4.14415227e-01  9.10079871e-01]
 [ 2.62305291e-01 -9.10724355e-01 -3.19025209e-01]
 [-6.91152360e-04 -9.84510839e-01 -1.75322363e-01]]


### Ransac
We will now attempt to use ransac to do a global registration of the two poinclouds.

Using the function [```o3d.pipelines.registration.registration_ransac_based_on_feature_matching```](http://www.open3d.org/docs/latest/python_api/open3d.pipelines.registration.registration_ransac_based_on_feature_matching.html) from open3d
    
    RANSACConvergenceCriteria. It defines the maximum number of RANSAC iterations and the confidence probability.
    The larger these two numbers are, the more accurate the result is, but also the more time the algorithm takes.

Try to find the transformation from r1 to r2.
Attempt with point to point and point to plane
```Python
point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
```

When using ransac focus on the arguments below the rest are optional
```Python
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point)
```

In [8]:
####
# Call RANSAC here
####

point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()

def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    
    distance_threshold = voxel_size * 1.5
    
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
   
    ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
                source_down,
                target_down,
                source_fpfh,
                target_fpfh,
                True, # mutual_filter
                distance_threshold, # max_correspondence_distance 
                o3d.pipelines.registration.TransformationEstimationPointToPoint(False), # estimation_method
#                 3, # ransac_n
#                 ([
#                 o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
#                 o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
#                 ]), # checkers
#                 o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
                )
    return ransac_result

ransac_result = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
print(ransac_result)
draw_registrations(source_down, target_down, ransac_result.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
RegistrationResult with fitness=2.312860e-01, inlier_rmse=4.313868e-02, and correspondence_set size of 241
Access transformation to get result.


## Exercises
### A)
    Can you get a decent transformation from r1 to r3?
### B)
    with the following checkers can you get better results from RANSAC? Try tweaking the parameters of them Can you make Point to Plane work do not spend too long on this if you cant skip it. (I was not able to get a good fit)

You can also try tweaking the voxel_size
```Python
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

cecker_list = [c0,c1,c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)
```


In [21]:
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

checker_list = [c0,c1,c2]

ransac_result2 = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, 
    source_fpfh, target_fpfh, 
    True,
    distance_threshold,
    point_to_point,
    checkers = checker_list)

print(ransac_result2)
draw_registrations(source_down, target_down, ransac_result2.transformation)

RegistrationResult with fitness=2.303263e-01, inlier_rmse=4.379523e-02, and correspondence_set size of 240
Access transformation to get result.
